# Threat actor profiling

## Preliminaries

Threat actors refer to cyber criminals and Advanced Persistent Threats (APTs) that have an interest on attacking an organization to obtain something of value to them. Valuable assets often include data they can sell for profit or operations they can hold for ransom.

To put in place effective defenses against these adversaries, we first need to understand who they are and how they operate.

### Baseline questions

- Which groups are likely to attack organization X?
- How will they likely attack X?
- What mitigations or detections can X create to defend against them?


## Prerequisites
Load MITRE ATT&CK Enterprise. You can use the ETL command in the SATRAP CLI or load the dump files included in this folder.

## Preparation
Set up the DB parameters and import the toolkit of SATRAP.

In [ ]:
from satrap.service.satrap_analysis import CTIanalysisToolbox
from satrap.commons.format_utils import format_dict, tabulate_groups, tabulate_stix_obj
from satrap.settings import TYPEDB_SERVER_ADDRESS, DB_NAME

satrap = CTIanalysisToolbox(TYPEDB_SERVER_ADDRESS, DB_NAME)

## Investigation

### Groups of interest

We start searching for identified groups that have some relation with a specific sector or whose description contains relevant keywords.

In [ ]:
groups = satrap.mitre_attack_groups(["manufacturing", "telecom"])
print(f"{len(groups)} groups found")
print(tabulate_groups(groups))

In [ ]:
group_ids = [group.group_id for group in groups]

In [ ]:
for group in group_ids:
    info = satrap.search_by_mitre_id(group)
    for data in info:
        print(tabulate_stix_obj(data))
    

### Techniques

Which techniques do these groups use?

In [ ]:
tech = satrap.techniques_used_by_groups(group_ids)
print(f"{len(tech)} techniques used by groups {group_ids}")
print(*tech,sep="\n")

Do they have techniques in common?

Learning this can help to prioritize those techniques that are more wide spread amongst our groups of interest.

In [ ]:
intersect = satrap.get_techniques_used_by_all(group_ids)
print(f"{len(intersect)} techniques used by groups {group_ids}")
print(format_dict(intersect))

- If we find techniques, let's get details about them ...

- Can we get insights using the inference engine?

Then, let's understand why these techniques are found to be used by all the groups.

In [ ]:
reason = satrap.explain_techniques_used_by_groups(group_ids, "...")
print(f"EXECUTED QUERY\n{reason.query}\n")
for tech in reason.explanations:
	print(format_dict(tech.as_json()), end="\n")

### Mitigations

Finally:

- Let's try to find some possible mitigations for the techniques we found ...

- What other mitigations can we consider against the groups of interest?

In [ ]:
satrap.related_mitigations("G0073")

- Why is _x_ mitigation relevant? (*Hint*: use `explain_if_related_mitigation`)